In [2]:

import sys
import re
import difflib
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

from config import (NUMBERS,
                                   NUMBERS3,
                                   NUMBERS4,
                                   NUMBERS4_RESULT,
                                   NUMBERS4_VECTORS,
                                   NUMBERS4_PATTERNS,
                                   NUMBERS4_PREDICTS)
import utils


%load_ext autoreload
%autoreload 2

##### NUMBERS4 で予測する

In [3]:
X = []
Y = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0

key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 3:
        if len(list(set(_y) & set(_v))) >= 3 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_format)
    # Y.append(_y)
    # X.append(_x)
    pass
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)
_sum = sum([key_0, key_1, key_2, key_3, key_4, key_5])

print(_format1)

前回:9個, 前々回:7個, 前回曜日:8個, 前々回曜日:6個, 前回六曜:8個, 前々回六曜:14個


In [7]:
X = []
Y = []
Z = []
LOCK = {}
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time]
    lock = []
    hook = []
    key_0 = 0
    key_1 = 0
    key_2 = 0
    key_3 = 0
    key_4 = 0
    key_5 = 0
    key_6 = 0
    key_7 = 0
    key_8 = 0
    key_9 = 0
    for _n in _x:
        if _time == _time:
            _h = list(_n)
            for _j in _h:
                if _j == '0':
                    key_0 += 1
                elif _j == '1':
                    key_1 += 1
                elif _j == '2':
                    key_2 += 1
                elif _j == '3':
                    key_3 += 1
                elif _j == '4':
                    key_4 += 1
                elif _j == '5':
                    key_5 += 1
                elif _j == '6':
                    key_6 += 1
                elif _j == '7':
                    key_7 += 1
                elif _j == '8':
                    key_8 += 1
                elif _j == '9':
                    key_9 += 1
            lock.append(list(_n))
    data = {
        "0": str(int(key_0) + 1),
        "1": str(int(key_1) + 1),
        "2": str(int(key_2) + 1),
        "3": str(int(key_3) + 1),
        "4": str(int(key_4) + 1),
        "5": str(int(key_5) + 1),
        "6": str(int(key_6) + 1),
        "7": str(int(key_7) + 1),
        "8": str(int(key_8) + 1),
        "9": str(int(key_9) + 1),
    }
    LOCK.update({ _time: [key_0, key_1, key_2, key_3, key_4, key_5, key_6, key_7, key_8, key_9, _y]})
    Z.append(data)
    X.append(lock)
    
lock = pd.DataFrame(LOCK)

In [8]:
LOCK

{'4964': [3, 1, 2, 1, 1, 8, 2, 3, 2, 1, '6817'],
 '4963': [2, 0, 3, 4, 1, 6, 2, 3, 3, 0, '1963'],
 '4962': [0, 3, 2, 6, 2, 2, 2, 3, 2, 2, '0555'],
 '4961': [1, 2, 1, 3, 3, 6, 1, 1, 4, 2, '2336'],
 '4960': [1, 1, 3, 2, 3, 3, 3, 3, 3, 2, '3155'],
 '4959': [4, 1, 2, 1, 1, 1, 2, 4, 6, 2, '5452'],
 '4958': [0, 2, 0, 4, 4, 1, 1, 3, 4, 5, '7278'],
 '4957': [1, 0, 0, 3, 2, 2, 6, 3, 4, 3, '8943'],
 '4956': [4, 1, 1, 4, 1, 1, 3, 3, 3, 3, '6894'],
 '4955': [2, 3, 2, 1, 2, 3, 3, 3, 4, 1, '6830'],
 '4954': [2, 3, 3, 4, 2, 2, 1, 3, 1, 3, '6807'],
 '4953': [1, 3, 1, 3, 2, 2, 3, 6, 1, 2, '7835'],
 '4952': [2, 4, 2, 2, 3, 3, 2, 2, 2, 2, '7693'],
 '4951': [2, 3, 3, 2, 2, 1, 1, 4, 1, 5, '0185'],
 '4950': [2, 3, 4, 2, 1, 2, 3, 4, 0, 3, '2154'],
 '4949': [1, 3, 1, 4, 3, 2, 3, 3, 1, 3, '2079'],
 '4948': [4, 2, 1, 3, 1, 2, 2, 3, 2, 4, '9473'],
 '4947': [2, 7, 4, 2, 1, 2, 1, 3, 1, 1, '4698'],
 '4946': [1, 5, 3, 3, 2, 2, 1, 3, 0, 4, '2309'],
 '4945': [2, 4, 0, 4, 1, 3, 4, 3, 1, 2, '1762'],
 '4944': [2, 6, 2, 3

In [6]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4_RESULT[_i][_time]
    _z = NUMBERS[_i][4]
    if _z == "先勝":
        Y.append(_y)
        X.append(_x)
        TIME.append(_time)

In [7]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x0 = NUMBERS4_RESULT[_i][_time][0]
    _x1 = NUMBERS4_RESULT[_i][_time][1]
    _x2 = NUMBERS4_RESULT[_i][_time][2]
    _x3 = NUMBERS4_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

# 20180618
# data = [float("1"), float("1"), float("1"), float("1")]
data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

22 272 71 1192


In [14]:
utils.get_predicts("4965", NUMBERS4_PREDICTS)

12753 8
11336 9
10202 10
9275 11
8502 12
7848 13
7287 14
6801 15
6376 16
6001 17
5668 18
5369 19
5101 20
4858 21
4637 22
4435 23
4251 24
4081 25
3924 26
3778 27
3643 28
3518 29
3400 30
3291 31
3188 32
3091 33
3000 34
2915 35
2834 36


In [11]:
utils.get_info("2018", "仏滅", "numbers4")

4949 2079
4945 1762
4938 5935
4924 3636
4920 6515
4918 8659
4904 1125
4900 8323
4897 6818
4893 2896
4879 5637
4877 6566
4873 3159
4859 5550
4856 6647
4852 2675
4848 8905
4835 5564
4831 1523
4827 5319


In [13]:
## １２：しも２kた、13:かみ2kた -> 7802